# Lecture 18: Mongo DB I

In [1]:
import json
import pymongo
import pprint

## Connect to Database on localhost

Nobel laureate data from Kaggle: https://www.kaggle.com/datasets/imdevskp/nobel-prize/data

In [2]:
client = pymongo.MongoClient('mongodb://localhost')
client.list_database_names()

['admin',
 'aggquery',
 'aggquerydb',
 'config',
 'local',
 'nobel_prizes',
 'tutorial']

In [3]:
# client.drop_database('nobel_prizes') # if already exists

Connect to an empty database called `nobel_prizes`

In [4]:
db = client.nobel_prizes
# db = client['nobel_prizes'] # also works
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nobel_prizes')

In [5]:
db.list_collection_names()

['prizes']

<br/><br/>Within this empty database, access an empty collection called `prizes`.

In [6]:
collection = db.prizes
# collection = db['prizes'] # also works
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nobel_prizes'), 'prizes')

`find_one()` returns None (i.e., displays nothing in notebooks)

In [7]:
db.prizes.find_one({})

{'_id': ObjectId('65371d7425624134ef010320'),
 'year': 2020,
 'category': 'chemistry',
 'laureates': [{'id': '991',
   'firstname': 'Emmanuelle',
   'surname': 'Charpentier',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'},
  {'id': '992',
   'firstname': 'Jennifer A.',
   'surname': 'Doudna',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'}]}

In [8]:
print(db.prizes.find_one({}))

{'_id': ObjectId('65371d7425624134ef010320'), 'year': 2020, 'category': 'chemistry', 'laureates': [{'id': '991', 'firstname': 'Emmanuelle', 'surname': 'Charpentier', 'motivation': '"for the development of a method for genome editing"', 'share': '2'}, {'id': '992', 'firstname': 'Jennifer A.', 'surname': 'Doudna', 'motivation': '"for the development of a method for genome editing"', 'share': '2'}]}


## Load data from JSON 

In [9]:
# run this cell to insert into the collection prizes
with open('data/prize.json', encoding='utf-8') as f:
    db.prizes.insert_many(json.loads(f.read()))

In [10]:
# the database and the collection are now both valid
client.list_database_names()

['admin',
 'aggquery',
 'aggquerydb',
 'config',
 'local',
 'nobel_prizes',
 'tutorial']

In [11]:
db.list_collection_names()

['prizes']

# Retrieval Queries

**Select** all field-value pairs to output.

In [12]:
db.prizes.find_one({})

{'_id': ObjectId('65371d7425624134ef010320'),
 'year': 2020,
 'category': 'chemistry',
 'laureates': [{'id': '991',
   'firstname': 'Emmanuelle',
   'surname': 'Charpentier',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'},
  {'id': '992',
   'firstname': 'Jennifer A.',
   'surname': 'Doudna',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'}]}

Pymongo is smart and avoids querying the **full collection** by instead returning an iterator (here, a Cursor):

In [13]:
db.prizes.find({})

It's not recommended, but you could force query the full collection by casting to list (e.g., `list(db.prizes.find({}))`).

Instead let's iterate over the collection a tiny bit so that we can see how to pretty print each document with `pprint`:

In [14]:
def pretty_print(output_collection, n_to_print=3):
    """
    note if n_to_print is -1, this print everything
    """
    for i, doc in enumerate(output_collection):
        pprint.pprint(doc)        # nicely formats each document
        if i+1 == n_to_print: return

In [15]:
output = db.prizes.find({})
pretty_print(output)

{'_id': ObjectId('65371d7425624134ef010320'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Emmanuelle',
                'id': '991',
                'motivation': '"for the development of a method for genome '
                              'editing"',
                'share': '2',
                'surname': 'Charpentier'},
               {'firstname': 'Jennifer A.',
                'id': '992',
                'motivation': '"for the development of a method for genome '
                              'editing"',
                'share': '2',
                'surname': 'Doudna'}],
 'year': 2020}
{'_id': ObjectId('65371d7425624134ef010321'),
 'category': 'economics',
 'laureates': [{'firstname': 'Paul',
                'id': '995',
                'motivation': '"for improvements to auction theory and '
                              'inventions of new auction formats"',
                'share': '2',
                'surname': 'Milgrom'},
               {'firstname': 'Robert',
 

# Selection (with Predicates)

In [16]:
# get a document that has the exact FV pair
db.prizes.find_one({"category": "chemistry"})

{'_id': ObjectId('65371d7425624134ef010320'),
 'year': 2020,
 'category': 'chemistry',
 'laureates': [{'id': '991',
   'firstname': 'Emmanuelle',
   'surname': 'Charpentier',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'},
  {'id': '992',
   'firstname': 'Jennifer A.',
   'surname': 'Doudna',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'}]}

In [17]:
# the comma functions as the "and" operator
db.prizes.find_one({"category": "chemistry", "year": 2020})

{'_id': ObjectId('65371d7425624134ef010320'),
 'year': 2020,
 'category': 'chemistry',
 'laureates': [{'id': '991',
   'firstname': 'Emmanuelle',
   'surname': 'Charpentier',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'},
  {'id': '992',
   'firstname': 'Jennifer A.',
   'surname': 'Doudna',
   'motivation': '"for the development of a method for genome editing"',
   'share': '2'}]}

## Dollar `$` Notation: Special MongoDB Keywords

(we defined the `pretty_print()` function earlier)

In [18]:
output = db.prizes.find({"$or": [{"category": "chemistry"}, {"year": 2020}]})

pretty_print(output, n_to_print=4)

{'_id': ObjectId('65371d7425624134ef010320'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Emmanuelle',
                'id': '991',
                'motivation': '"for the development of a method for genome '
                              'editing"',
                'share': '2',
                'surname': 'Charpentier'},
               {'firstname': 'Jennifer A.',
                'id': '992',
                'motivation': '"for the development of a method for genome '
                              'editing"',
                'share': '2',
                'surname': 'Doudna'}],
 'year': 2020}
{'_id': ObjectId('65371d7425624134ef010321'),
 'category': 'economics',
 'laureates': [{'firstname': 'Paul',
                'id': '995',
                'motivation': '"for improvements to auction theory and '
                              'inventions of new auction formats"',
                'share': '2',
                'surname': 'Milgrom'},
               {'firstname': 'Robert',
 

In [19]:
output = db.prizes.find({"year": {"$gt": 2018}}, skip=6) # skip a bit down the collection just because

pretty_print(output)

{'_id': ObjectId('65371d7425624134ef010326'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'John',
                'id': '976',
                'motivation': '"for the development of lithium-ion batteries"',
                'share': '3',
                'surname': 'Goodenough'},
               {'firstname': 'M. Stanley',
                'id': '977',
                'motivation': '"for the development of lithium-ion batteries"',
                'share': '3',
                'surname': 'Whittingham'},
               {'firstname': 'Akira',
                'id': '978',
                'motivation': '"for the development of lithium-ion batteries"',
                'share': '3',
                'surname': 'Yoshino'}],
 'year': 2019}
{'_id': ObjectId('65371d7425624134ef010327'),
 'category': 'economics',
 'laureates': [{'firstname': 'Abhijit',
                'id': '982',
                'motivation': '"for their experimental approach to alleviating '
                              '

## Dot `.` Notation: Traverse Trees

In [20]:
output = db.prizes.find({"laureates.0.surname": "Curie"})
pretty_print(output)

{'_id': ObjectId('65371d7425624134ef010575'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Marie',
                'id': '6',
                'motivation': '"in recognition of her services to the '
                              'advancement of chemistry by the discovery of '
                              'the elements radium and polonium, by the '
                              'isolation of radium and the study of the nature '
                              'and compounds of this remarkable element"',
                'share': '1',
                'surname': 'Curie'}],
 'year': 1911}
{'_id': ObjectId('6538178131ad5e3e548b9456'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Marie',
                'id': '6',
                'motivation': '"in recognition of her services to the '
                              'advancement of chemistry by the discovery of '
                              'the elements radium and polonium, by the '
                              'isolatio

In [21]:
output = db.prizes.find({"laureates.1.surname": "Curie"})
pretty_print(output)

{'_id': ObjectId('65371d7425624134ef0105a0'),
 'category': 'physics',
 'laureates': [{'firstname': 'Henri',
                'id': '4',
                'motivation': '"in recognition of the extraordinary services '
                              'he has rendered by his discovery of spontaneous '
                              'radioactivity"',
                'share': '2',
                'surname': 'Becquerel'},
               {'firstname': 'Pierre',
                'id': '5',
                'motivation': '"in recognition of the extraordinary services '
                              'they have rendered by their joint researches on '
                              'the radiation phenomena discovered by Professor '
                              'Henri Becquerel"',
                'share': '4',
                'surname': 'Curie'},
               {'firstname': 'Marie',
                'id': '6',
                'motivation': '"in recognition of the extraordinary services '
                  

In [22]:
output = db.prizes.find({"laureates.surname": "Curie"})
pretty_print(output)

{'_id': ObjectId('65371d7425624134ef010575'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Marie',
                'id': '6',
                'motivation': '"in recognition of her services to the '
                              'advancement of chemistry by the discovery of '
                              'the elements radium and polonium, by the '
                              'isolation of radium and the study of the nature '
                              'and compounds of this remarkable element"',
                'share': '1',
                'surname': 'Curie'}],
 'year': 1911}
{'_id': ObjectId('65371d7425624134ef0105a0'),
 'category': 'physics',
 'laureates': [{'firstname': 'Henri',
                'id': '4',
                'motivation': '"in recognition of the extraordinary services '
                              'he has rendered by his discovery of spontaneous '
                              'radioactivity"',
                'share': '2',
                'surname': 

# Projection

In [23]:
output = db.prizes.find({}, {"year": 1, "category": 1})

pretty_print(output)

{'_id': ObjectId('65371d7425624134ef010320'),
 'category': 'chemistry',
 'year': 2020}
{'_id': ObjectId('65371d7425624134ef010321'),
 'category': 'economics',
 'year': 2020}
{'_id': ObjectId('65371d7425624134ef010322'),
 'category': 'literature',
 'year': 2020}


<br/><br/><br/><br/><br/>

## Projection Exercise

In [24]:
output = db.prizes.find({}, {"year": 1, "category": 1, "_id": 0})

pretty_print(output)

{'category': 'chemistry', 'year': 2020}
{'category': 'economics', 'year': 2020}
{'category': 'literature', 'year': 2020}


In [25]:
output = db.prizes.find({}, {"year": 0, "category": 0})

pretty_print(output)

{'_id': ObjectId('65371d7425624134ef010320'),
 'laureates': [{'firstname': 'Emmanuelle',
                'id': '991',
                'motivation': '"for the development of a method for genome '
                              'editing"',
                'share': '2',
                'surname': 'Charpentier'},
               {'firstname': 'Jennifer A.',
                'id': '992',
                'motivation': '"for the development of a method for genome '
                              'editing"',
                'share': '2',
                'surname': 'Doudna'}]}
{'_id': ObjectId('65371d7425624134ef010321'),
 'laureates': [{'firstname': 'Paul',
                'id': '995',
                'motivation': '"for improvements to auction theory and '
                              'inventions of new auction formats"',
                'share': '2',
                'surname': 'Milgrom'},
               {'firstname': 'Robert',
                'id': '996',
                'motivation': '"for imp

In [26]:
output = db.prizes.find({}, {"year": 0, "category": 1})

pretty_print(output)

OperationFailure: Cannot do inclusion on field category in exclusion projection, full error: {'ok': 0.0, 'errmsg': 'Cannot do inclusion on field category in exclusion projection', 'code': 31253, 'codeName': 'Location31253'}

In [27]:
db.prizes.find_one({"overallMotivation": {"$exists": 1}})

{'_id': ObjectId('65371d7425624134ef01032a'),
 'year': 2019,
 'category': 'physics',
 'overallMotivation': '"for contributions to our understanding of the evolution of the universe and Earth’s place in the cosmos"',
 'laureates': [{'id': '973',
   'firstname': 'James',
   'surname': 'Peebles',
   'motivation': '"for theoretical discoveries in physical cosmology"',
   'share': '2'},
  {'id': '974',
   'firstname': 'Michel',
   'surname': 'Mayor',
   'motivation': '"for the discovery of an exoplanet orbiting a solar-type star"',
   'share': '4'},
  {'id': '975',
   'firstname': 'Didier',
   'surname': 'Queloz',
   'motivation': '"for the discovery of an exoplanet orbiting a solar-type star"',
   'share': '4'}]}

# Sorting and Limits

In [28]:
output = (db.prizes.find({"category": "peace"},
               {"_id": 0, "category": 1, "year": 1,
                "laureates.firstname": 1, "laureates.surname": 1})
          .sort("year")
         )

pretty_print(output)

{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}
{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}
{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}


In [29]:
output = (db.prizes.find({"category": "peace"},
               {"_id": 0, "category": 1, "year": 1,
                "laureates.firstname": 1, "laureates.surname": 1})
          .sort("year", -1)
         )

pretty_print(output)

{'category': 'peace',
 'laureates': [{'firstname': 'World Food Programme'}],
 'year': 2020}
{'category': 'peace',
 'laureates': [{'firstname': 'World Food Programme'}],
 'year': 2020}
{'category': 'peace',
 'laureates': [{'firstname': 'World Food Programme'}],
 'year': 2020}


In [30]:
output = (db.prizes.find({"category": "peace"},
               {"_id": 0, "category": 1, "year": 1,
                "laureates.firstname": 1, "laureates.surname": 1})
          .sort([("year", 1), ("category", -1)])
         )

pretty_print(output)

{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}
{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}
{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}


In [31]:
output = (db.prizes.find({"category": "peace"},
               {"_id": 0, "category": 1, "year": 1,
                "laureates.firstname": 1, "laureates.surname": 1})
          .sort([("year", 1), ("category", -1)])
          .limit(2)
         )

pretty_print(output)

{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}
{'category': 'peace',
 'laureates': [{'firstname': 'Henry', 'surname': 'Dunant'},
               {'firstname': 'Frédéric', 'surname': 'Passy'}],
 'year': 1901}


# Aggregation Queries

Zips JSON from the MongoDB Aggregation Tutorial: https://www.mongodb.com/docs/manual/tutorial/aggregation-zip-code-data-set/

## Load `zips.json` into new `local.zips`

For the sake of simplicity, we'll make a new collection `zips` in a new `aggquerydb` database.

In [32]:
# reimport/redefine as needed
import json
import pymongo
import pprint

def pretty_print(output_collection, n_to_print=3):
    """
    note if n_to_print is -1, this print everything
    """
    for i, doc in enumerate(output_collection):
        pprint.pprint(doc)        # nicely formats each document
        if i+1 == n_to_print: return

In [33]:
client = pymongo.MongoClient('mongodb://localhost')
client.list_database_names()

['admin',
 'aggquery',
 'aggquerydb',
 'config',
 'local',
 'nobel_prizes',
 'tutorial']

In [34]:
# run this cell to make the new collection
# and insert zipcode documents

client.drop_database('aggquerydb') # if already exists
db = client.aggquerydb

with open('data/zips.json', encoding='utf-8') as f:
    for line in f.readlines():
        db.zips.insert_one(json.loads(line))

In [35]:
db.zips.count_documents({})

29353

In [36]:
output = db.zips.aggregate( [    
{ "$group": { "_id": "$state",  
  "totalPop": { "$sum": "$pop" } } },    
{ "$match": { "totalPop":
            { "$gte": 15000000 } } }, 
{ "$sort" : { "totalPop" : -1 } }
] )

pretty_print(output)

{'_id': 'CA', 'totalPop': 29754890}
{'_id': 'NY', 'totalPop': 17990402}
{'_id': 'TX', 'totalPop': 16984601}


# Aggregation Queries: Unwind/Lookup

Make a new collection, `inventory`.

In [37]:
# stay in the aggquerydb database
db = client.aggquerydb

In [38]:
db.inventory.insert_many( [    
{ "item": "journal",
 "tags": ["blank", "red"],
 "dim": [ 14, 21 ],
 "instock": [ { "loc": "A", "qty": 5 }, { "loc": "C", "qty": 15 } ]
},    
{ "item": "notebook",
 "tags": ["red", "blank"],
 "dim": [ 14, 21 ],
 "instock": [ { "loc": "C", "qty": 5 } ]
},    
{ "item": "paper",
 "tags": ["red", "blank", "plain"],
 "dim": [ 14, 21 ],
 "instock": [ { "loc": "A", "qty": 60 }, { "loc": "B", "qty": 15 } ]
},    
{ "item": "planner",
 "tags": ["blank", "red"],
 "dim": [ 22.85, 30 ] ,
 "instock": [ { "loc": "A", "qty": 40 }, { "loc": "B", "qty": 5 } ]
},    
{ "item": "postcard",
 "tags": ["blue"],
 "dim": [ 10, 15.25 ],
 "instock": [ { "loc": "B", "qty": 15 }, { "loc": "C", "qty": 35 } ]
}
]);

In [39]:
output = db.inventory.aggregate( [ 
{ "$unwind" : "$tags" }, 
{ "$project" : {"_id" : 0, "instock": 0}}  
] )

pretty_print(output, n_to_print=-1)

{'dim': [14, 21], 'item': 'journal', 'tags': 'blank'}
{'dim': [14, 21], 'item': 'journal', 'tags': 'red'}
{'dim': [14, 21], 'item': 'notebook', 'tags': 'red'}
{'dim': [14, 21], 'item': 'notebook', 'tags': 'blank'}
{'dim': [14, 21], 'item': 'paper', 'tags': 'red'}
{'dim': [14, 21], 'item': 'paper', 'tags': 'blank'}
{'dim': [14, 21], 'item': 'paper', 'tags': 'plain'}
{'dim': [22.85, 30], 'item': 'planner', 'tags': 'blank'}
{'dim': [22.85, 30], 'item': 'planner', 'tags': 'red'}
{'dim': [10, 15.25], 'item': 'postcard', 'tags': 'blue'}


In [40]:
output = db.inventory.aggregate( [ 
	{ "$unwind" : "$instock" },  
	{ "$group" : { "_id" : "$item", "totalqty" : {"$sum" : "$instock.qty"}}} 
] )

pretty_print(output, n_to_print=-1)

{'_id': 'journal', 'totalqty': 20}
{'_id': 'planner', 'totalqty': 45}
{'_id': 'postcard', 'totalqty': 50}
{'_id': 'paper', 'totalqty': 75}
{'_id': 'notebook', 'totalqty': 5}


In [41]:
output = db.inventory.aggregate( [ 
{ "$lookup" : {
    "from" : "inventory",
    "localField": "instock.loc", 
    "foreignField": "instock.loc", 
    "as":"otheritems"}
},  
{ "$project" :
    {"_id" : 0, "tags" : 0, "dim" : 0}
} ] )

pretty_print(output, n_to_print=1)

{'instock': [{'loc': 'A', 'qty': 5}, {'loc': 'C', 'qty': 15}],
 'item': 'journal',
 'otheritems': [{'_id': ObjectId('6538194925af2742ae5cacc3'),
                 'dim': [14, 21],
                 'instock': [{'loc': 'A', 'qty': 5}, {'loc': 'C', 'qty': 15}],
                 'item': 'journal',
                 'tags': ['blank', 'red']},
                {'_id': ObjectId('6538194925af2742ae5cacc4'),
                 'dim': [14, 21],
                 'instock': [{'loc': 'C', 'qty': 5}],
                 'item': 'notebook',
                 'tags': ['red', 'blank']},
                {'_id': ObjectId('6538194925af2742ae5cacc5'),
                 'dim': [14, 21],
                 'instock': [{'loc': 'A', 'qty': 60}, {'loc': 'B', 'qty': 15}],
                 'item': 'paper',
                 'tags': ['red', 'blank', 'plain']},
                {'_id': ObjectId('6538194925af2742ae5cacc6'),
                 'dim': [22.85, 30],
                 'instock': [{'loc': 'A', 'qty': 40}, {'loc': 'B', 'q